In [1]:
from neuralogic.nn import get_evaluator
from neuralogic.core import R, Template, V, Settings, Transformation, Combination, Aggregation
from neuralogic.core.constructs.function.tree import FunctionContainer
from neuralogic.core.settings import Settings
from neuralogic.dataset import Dataset
from neuralogic.optim import SGD

C:\Users\kraus\Plocha\BAK\PyNeuraLogicFork\neuralogic\core\builder\builder.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
F = FunctionContainer()     # new but necessary feature, necessary to use both round and square brackets for same purpose
train_dataset = Dataset()
template = Template()
settings = Settings(optimizer=SGD(), epochs=1)

train_dataset.add_example([R.b[3], R.c[5]])

V následující buňce je ukázka funkční nové syntaxe, kterou jsi po mně chtěl v tom nedávném emailu. Nyní fungují tyto obě varianty stejně.

Poměrně obtížné bylo zajistit, aby hranaté závorky fungovaly stejně jako kulaté a zápis aby byl přehlednější. To se mi nakonec povedlo za cenu, že třída FunctionContainer z modulu neuralogic.core.constructs.function.tree je trochu složitější a hůře čitelná.

POZOR: Pokud odmažeme komentář tak v tuto chvíli hodí kod chybu. Strom syntaxe je vytvořen správně, ale mám někde bug v evaluaci. Více to rozepisuji v další sekci.

In [3]:
# 1st option
#template += (R.message2(V.X)[5,10] <= (x:= R.message1(V.Y)[10,20], y:= R.edge(V.Y, V.X))) >> F.avg(F.relu(x + y))

# 2nd option - round brackets
#template += (R.message2(V.X)[5,10] <= F.avg(F.relu(R.message1(V.Y)[10,20] + R.edge(V.Y, V.X))))

# 2nd option - both round and square brackets
#template += (R.message2(V.X)[5,10] <= F.avg[F.relu(R.message1(V.Y)[10,20] + R.edge(V.Y, V.X))])

V této další sekci jsem zkoušel přímo kopírovat tu vaši strukturu [kombinace, transformace, agregace], aby moje syntaxe volala v pythonu v podstatě to stejné.
Myslím, že jsem půl krůčku od řešení, ale mám tam někde menší bug, který se mi zatím nepodařilo najít.

Každopádně volání "F.avg[F.identity(x * y)]" mi nyní vrací úplně identický výraz jako "[Combination.PRODUCT, Transformation.IDENTITY, Aggregation.AVG]", a bude fungovat i pro jiné kombinace těchto funkcí. Takto vytvořená metadata by se pak pravidlu měla předat úplně stejně jako tomu bylo dříve, ale asi jsem tam něco opomenul. Myslím ale, že musí jít o nějakou malichernost.

In [4]:
# Original working syntax
#template += (R.a <= (R.b, R.c)) | [Combination.PRODUCT, Transformation.IDENTITY, Aggregation.AVG]

# New currently not working syntax
template += (R.a <= (x:=R.b, y:= R.c)) >> F.avg[F.identity(x * y)]

printing operations
['*', 'identity', 'avg_a']
printing metadata
[<neuralogic.core.constructs.function.function.Combination object at 0x000001CD0D6A44C0>, <neuralogic.core.constructs.function.function.Transformation object at 0x000001CD0D687EC0>, <neuralogic.core.constructs.function.function.Aggregation object at 0x000001CD0D4A3140>]


Následuje neupravený zbytek kodu.

In [5]:
train_dataset.add_queries([R.a[3]])

neuralogic_evaluator = get_evaluator(template, settings)

build_dataset = neuralogic_evaluator.build_dataset(train_dataset)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

java.util.InputMismatchException: java.util.InputMismatchException: Query: [a] was not matched anywhere in the ground network - Cannot calculate its output!

In [ ]:
build_dataset[0].draw()